# Jupyter (iPython) Notebook to reproduce the table values in TB-2018-001

Dependencies
------------

    numpy (http://www.numpy.org/)
    colour-science (http://www.colour-science.org/)
    iPython (https://ipython.org/)
    pandas (https://pandas.pydata.org/)

Copyright Academy of Motion Picture Arts and Sciences

# Setup Notebook

In [1]:
# Imports for Calculations
import colour
import numpy as np

In [2]:
# Imports for Display of Tables
from IPython.display import HTML, display
import pandas as pd

In [3]:
# Turn off any warnings
import warnings
warnings.simplefilter("ignore")

## Table 1 - CCT of canonical CIE daylight illuminants

Specify nominal CCT values for canonical CIE Daylight Illuminants

In [4]:
cct_pre68 = np.array([5000, 5500, 6500, 7500])

Define the correction factor

In [5]:
c_factor = 1.4388 / 1.4380

Apply the correction factor to determine the actual CCT values for the daylight illuminants

In [6]:
cct_post68 = cct_pre68 * c_factor

In [7]:
# Print the table
df_T1 = pd.DataFrame( np.c_[cct_pre68, cct_post68, cct_post68], 
                      index = ['D50', 'D55', 'D65', 'D75'], 
                      columns = ['CCT before 1968', 
                                 'CCT Current (round to 3 decimal places)', 
                                 'CCT Current (round to 0 decimal places)'] )

df_T1.iloc[:, 0] = df_T1.iloc[:, 0].map('{:,.0f} K'.format)
df_T1.iloc[:, 1] = df_T1.iloc[:, 1].map('{:,.3f} K'.format)
df_T1.iloc[:, 2] = df_T1.iloc[:, 2].map('{:,.0f} K'.format)

display(df_T1)

,CCT before 1968,CCT Current (round to 3 decimal places),CCT Current (round to 0 decimal places)
D50,"5,000 K","5,002.782 K","5,003 K"
D55,"5,500 K","5,503.060 K","5,503 K"
D65,"6,500 K","6,503.616 K","6,504 K"
D75,"7,500 K","7,504.172 K","7,504 K"


## CCT of "D60"

In [8]:
# Calculate the CCT of a 'D60' CIE Daylight Illuminant assuming the name follows the canonical illuminants
cct_d60 = 6000 * c_factor

In [9]:
# Print the result
df_cctd60 = pd.DataFrame( cct_d60, 
                          index = ['D60'], 
                          columns = ['CCT'] )

display(df_cctd60)

,CCT
D60,6003.337969


## Table 2 - CIE xy chromaticity coordinates rounded to 5 decimal places

In [10]:
# Calculate the ACES white point CIE xy values
xy_aces = colour.models.ACES_2065_1_COLOURSPACE.whitepoint

# Calculate the CIE xy values for CIE Daylight of 6000K
xy_d6000 = colour.CCT_to_xy(6000, method='CIE Illuminant D Series')

# Calculate the CIE xy values for CIE Daylight of 'D60'
xy_d60 = colour.CCT_to_xy(cct_d60 , method='CIE Illuminant D Series')

# Place in an array
xy = np.array([xy_aces,xy_d6000,xy_d60])

In [11]:
# Print the table
df_T2 = pd.DataFrame( xy, 
                      index = ['ACES White Point', 'CIE Daylight 6000K', 'CIE Daylight 6003K'], 
                      columns = ['CIE x', 'CIE y'] )

pd.set_option('precision', 5)

display(df_T2)

,CIE x,CIE y
ACES White Point,0.32168,0.33767
CIE Daylight 6000K,0.32169,0.33780
CIE Daylight 6003K,0.32163,0.33774


## Table 3 - CIE u′v′ chromaticity coordinates and ∆u′v′ from the ACES white point rounded to 5 decimal places

In [12]:
# Calculate the CIE 1976 UCS u'v' values
uvp = colour.xy_to_Luv_uv(xy)

# Calculate the ∆u′v′ from the ACES white point
delta_uvp = colour.delta_E(uvp[0,:], uvp, method='cie1976')

In [13]:
# Print the table
df_T3 = pd.DataFrame( np.c_[uvp, delta_uvp], 
                      index = ['ACES White Point', 'CIE Daylight 6000K', 'CIE Daylight 6003K'], 
                      columns = ['CIE u\'', 'CIE v\'', '∆u\'v\''] )

pd.set_option('precision', 5)

display(df_T3)

,CIE u',CIE v',∆u'v'
ACES White Point,0.20078,0.47421,0.00000
CIE Daylight 6000K,0.20074,0.47427,0.00008
CIE Daylight 6003K,0.20072,0.47423,0.00007


## Table 4 - Estimation of the CCT of the ACES white point rounded to 2 decimal places

In [14]:
# Calculate the CIE 1960 UCS uv values
uv = colour.xy_to_UCS_uv(xy)

# Calculate the CCT from the chromaticity coordinates using various methods
cct_robertson = colour.uv_to_CCT(uv[0,:],'Robertson 1968')
cct_hernandez = colour.xy_to_CCT(xy[0,:],'Hernandez 1999')
cct_ohno = colour.uv_to_CCT(uv[0,:], 'Ohno 2013')
cct_mccamy = colour.xy_to_CCT(xy[0,:],'McCamy 1992')

In [15]:
# Print the table
df_T4 = pd.DataFrame( np.c_[cct_robertson[0], cct_hernandez, cct_ohno[0], cct_mccamy].T, 
                      index = ['Robertson', 'Hernandez-Andres', 'Ohno', 'McCamy'], 
                      columns = ['ACES white point CCT'] )

df_T4.index.name = 'CCT Estimation Method'

pd.set_option('precision', 2)

display(df_T4)

,ACES white point CCT
CCT Estimation Method,
Robertson,5998.98
Hernandez-Andres,5997.26
Ohno,6000.04
McCamy,6000.41


## Table 5 - Chromaticity difference from projected LAD patch

In [16]:
# Specify the CIE xy values of a projected LAD patch
xy_lad = np.array([0.32170, 0.33568])

# Daylight range from 5500 K to 6500 K in 100 K increments over which to calculate the differences
cct_dseries = np.arange(5500,6600,100)

# Calculate the CIE xy chromaticity coordinates of CIE Daylight
xy_dseries = colour.CCT_to_xy(cct_dseries, method='CIE Illuminant D Series')

# Calculate the CIE UCS 1976 u'v'
uvp_lad = colour.xy_to_Luv_uv(xy_lad)
uvp_dseries = colour.xy_to_Luv_uv(xy_dseries)

# Calculate the ∆u′v′ between the LAD and the CIE Daylight chromaticity coordinates
delta_uvp_lad = colour.delta_E(uvp_lad, uvp_dseries, method='cie1976')

In [17]:
# Print the table
df_T5 = pd.DataFrame( delta_uvp_lad.T, 
                      index = cct_dseries, 
                      columns =  ['∆u\'v\' from LAD chromaticity'])

df_T5.index.name = 'CCT Estimation Method'

pd.set_option('precision', 6)

display(df_T5)

,∆u'v' from LAD chromaticity
CCT Estimation Method,
5500,0.008183
5600,0.006619
5700,0.005112
5800,0.003676
5900,0.002354
6000,0.001360
6100,0.001448
6200,0.002442
6300,0.003627
